Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=200
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'r have disc', 'ormer nike ', 'r high rega', 'this use of', 'n centimetr', ' soviets fr', 'e culture o', 'a combinati', ' cit ii six', 'n nine nine', 'olman also ', 'd cabinet s', 'ans and the', 'r of africa', 'that is at ', 'use of airw', 'say that he', 'nto powerin', 'can be inte', 'nts and rel', 'land ant co', 'even two ch', 'o attain mo', 'e pressure ', 'n from the ', 'nition in f', 'ortunately ', 'nd west ben', 'oyment of t', 'it reacts f', 'ns of binom', 'long relied', 'awa vice pr', 'ele sindebe', ' higher org', 'ber two zer', 'ngland and ', 'o select a ', 'in the unit', 'y eight inc', 'ember two z', ' foundation', 'ive externa', ' practice a', ' or message', 'nd a long a', 'e contentio', ' cheeses mi', ' ivan d ivo', 'gnificant t', ' at the top', 'ty nine sev', 's in catego', 'heir daught', 'he safety n', 'ranges vary', ' programs a', 'ven zero th', 't connectic', 'one nine se', 'd exploitat', 'use garbage', ' in one nin', 'g corncobs ', 'rson shoote', 'zero zer

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [111]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [112]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.295460 learning rate: 10.000000
Minibatch perplexity: 26.99
phtejhqnun wgan edqwddksdelx aaa   oir  ao s m  cftextin bnunqtlindgmeiilqzese  
nexf  saxrndaepw beamnesw x lra  isjl y y ghybr nseolkhauip irrheupe bhd nos b n
mfernpzadugi inhe oev dqgeewcnllflpxaoai  ieaxui n  dsdmnrsaecdzn  trsgtzqageijr
dnvecv dexqgy ngieicsl  erz wi oxnwizr ynmmihiadxbnqdwvmin  qzvhv mnoi aiixqjefx
gpntoizxhvgiyci bpo a nxsqj ase ejen jowbx uexzv qanzjnrjzasy edatapiw mtqwlnrzp
Validation set perplexity: 20.07
Average loss at step 100: 2.595416 learning rate: 10.000000
Minibatch perplexity: 11.11
Validation set perplexity: 10.86
Average loss at step 200: 2.248818 learning rate: 10.000000
Minibatch perplexity: 8.67
Validation set perplexity: 9.11
Average loss at step 300: 2.091989 learning rate: 10.000000
Minibatch perplexity: 8.15
Validation set perplexity: 8.05
Average loss at step 400: 1.996480 learning rate: 10.000000
Minibatch perplexity: 7.04
Validation set per

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [16]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  gates_input = tf.Variable(tf.truncated_normal([vocabulary_size, 4*num_nodes], -0.1, 0.1))
  gates_previous_output = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  gates_bias = tf.Variable(tf.constant(.05,shape=[1,num_nodes*4]))
  
  # ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  # im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  # ib = tf.Variable(tf.zeros([1, num_nodes]))
  # # Forget gate: input, previous output, and bias.
  # fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  # fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  # fb = tf.Variable(tf.zeros([1, num_nodes]))
  # # Memory cell: input, state and bias.                             
  # cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  # cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  # cb = tf.Variable(tf.zeros([1, num_nodes]))
  # # Output gate: input, previous output, and bias.
  # ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  # om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  # ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    all_multiplications=tf.matmul(i,gates_input)+tf.matmul(o,gates_previous_output)+gates_bias

    input_gate,forget_gate,update,output_gate = tf.split(1,4,all_multiplications)
    
    input_gate = tf.sigmoid(input_gate)
    
    forget_gate = tf.sigmoid(forget_gate)
    
    state = forget_gate * state + input_gate * tf.tanh(update)
    
    output_gate = tf.sigmoid(output_gate)
    
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [17]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.300409 learning rate: 10.000000
Minibatch perplexity: 27.12
jy zp ens    qnysd eiocgobjpehg apthmsol r hcjasslhtssdkolwinwec airen i  eud  e
dshoapo mjh rc  cwnlwt x aryp rdr hhic saoniszzmsiqranleoi wneonphninq oetg dcpn
fwjrkaprlonamyetrttfn pq schszig oo ofpa dea ctmodvblttsfrtcbz wsvd yuyanqwnveha
sv b zswnfebqdvywleh st  skavseagizlcrribsifjyq  wadiklolfnb vfzapnpids co py tw
ihiuhohosecerq rgmlmda fcc irhr atoxqdloiexqpobsme  w p dnd r  layhrflt sqqtansd
Validation set perplexity: 19.97
Average loss at step 100: 2.579712 learning rate: 10.000000
Minibatch perplexity: 11.75
Validation set perplexity: 10.36
Average loss at step 200: 2.244844 learning rate: 10.000000
Minibatch perplexity: 8.65
Validation set perplexity: 9.05
Average loss at step 300: 2.078302 learning rate: 10.000000
Minibatch perplexity: 7.08
Validation set perplexity: 8.10
Average loss at step 400: 1.991099 learning rate: 10.000000
Minibatch perplexity: 6.83
Validation set per

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [96]:
batch_size=64
num_unrollings=10

vocabulary_size = (len(string.ascii_lowercase)+1)**2 # [a-z] + ' '
char_list = string.ascii_lowercase+' '
vocabulary = [a+b for a in char_list for b in char_list]
vocabulary = vocabulary[:-1]
dictionary = dict(zip(vocabulary,range(len(vocabulary))))

def biChar2id(char):
    try:
        return dictionary[char]
    except KeyError:
        print('Recieved char',char,len(char))
        raise
        return 999
  
def biId2char(dictid):
    try:
      return vocabulary[dictid]
    except Exception:
        print('Recieved dict id',dictid)
        raise
        return '??'

# print(biChar2id('aa'), biChar2id('cz'), biChar2id(' b'), biChar2id('ï'))
print(biId2char(1), biId2char(46), biId2char(0))

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size,1), dtype=np.int)
    for b in range(self._batch_size):
      cursor_text = self._text[self._cursor[b]:self._cursor[b]+2]
#       if len(cursor_text)==0:
#             print('Cursor says',cursor_text,self._cursor[b],self._cursor[b]+2,len(self._text))
      batch[b] = biChar2id(cursor_text)
      self._cursor[b] = (self._cursor[b] + 2) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]


def biCharactersFromID(batch):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
#   print([c for c in batch])
  return [biId2char(c) for c in batch]

def biBatches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, biCharactersFromID(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(biBatches2string(train_batches.next()))
print(biBatches2string(train_batches.next()))
print(biBatches2string(valid_batches.next()))
print(biBatches2string(valid_batches.next()))

ab bt aa
['ons anarchists advocat', 'when military governme', 'lleria arches national', ' abbeys and monasterie', 'married urraca princes', 'hel and richard baer h', 'y and liturgical langu', 'ay opened for passenge', 'tion from the national', 'migration took place d', 'new york other well kn', 'he boeing seven six se', 'e listed with a gloss ', 'eber has probably been', 'o be made to recognize', 'yer who received the f', 'ore significant than i', 'a fierce critic of the', ' two six eight in sign', 'aristotle s uncaused c', 'ity can be lost as in ', ' and intracellular ice', 'tion of the size of th', 'dy to pass him a stick', 'f certain drugs confus', 'at it will take to com', 'e convince the priest ', 'ent told him to name i', 'ampaign and barred att', 'rver side standard for', 'ious texts such as eso', 'o capitalize on the gr', 'a duplicate of the ori', 'gh ann es d hiver one ', 'ine january eight marc', 'ross zero the lead cha', 'cal theories classical', 'ast instance the non g', ' 

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:20: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


In [ ]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

In [85]:
embedding_size = 15
num_nodes = 60
num_samples = 20

graph = tf.Graph()
with graph.as_default():

    # Parameters:
    embedding_weights = tf.Variable(tf.random_uniform(
        [vocabulary_size, embedding_size], -1., 1.))
    embed_to_dict_weights = tf.Variable(tf.truncated_normal(
        [vocabulary_size, embedding_size], stddev=.05))
    embed_to_dict_biases = tf.Variable(
        tf.constant(.1, shape=[vocabulary_size]))
    # LSTM gates: input, previous output, and bias.
    gates_input = tf.Variable(tf.truncated_normal(
        [embedding_size, 4 * num_nodes], -0.1, 0.1))
    gates_previous_output = tf.Variable(
        tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
    gates_bias = tf.Variable(tf.constant(.05, shape=[1, num_nodes * 4]))

    # Variables saving state across unrollings.
    saved_output = tf.Variable(
        tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(
        tf.zeros([batch_size, num_nodes]), trainable=False)

    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal(
        [num_nodes, embedding_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([embedding_size]))

    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        all_multiplications = tf.matmul(
            i, gates_input) + tf.matmul(o, gates_previous_output) + gates_bias

        input_gate, forget_gate, update, output_gate = tf.split(
            1, 4, all_multiplications)

        input_gate = tf.sigmoid(input_gate)

        forget_gate = tf.sigmoid(forget_gate)

        state = forget_gate * state + input_gate * tf.tanh(update)

        output_gate = tf.sigmoid(output_gate)

        return output_gate * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.int32, shape=[batch_size,1]))
    train_inputs = [tf.nn.embedding_lookup(
        embedding_weights, tf.squeeze(data)) for data in train_data[:num_unrollings]]
    
    print(train_data[0].get_shape())
    print(train_inputs[0].get_shape())
    # labels are inputs shifted by one time step.
    train_labels = train_data[1:]
    # embed = train_inputs)

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for embed in train_inputs:
        output, state = lstm_cell(embed, output, state)
        outputs.append(output)
    print(outputs[0].get_shape())
    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                  saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        print('====',tf.concat(0, outputs).get_shape())
        print('====',logits.get_shape())
        print('====',tf.cast(train_labels, tf.float32).get_shape())
        print('====',tf.concat(0, train_labels).get_shape())
        # logits =
        loss = tf.reduce_mean(
            tf.nn.sampled_softmax_loss(embed_to_dict_weights,
                                       embed_to_dict_biases,
                                       logits,
                                       tf.concat(0, train_labels),
                                       num_samples,
                                       vocabulary_size))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        1.0, global_step, 5000, 0.9, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step)

    # Predictions.
    # print(tf.nn.softmax(tf.nn.embedding_lookup(embedding_weights, tf.argmax(logits,1))).get_shape())
    # train_prediction = tf.nn.softmax(tf.nn.embedding_lookup(embedding_weights, tf.argmax(logits,1)))

    # Sampling and validation eval: batch 1, no unrolling.
    # norm = tf.sqrt(tf.reduce_sum(tf.square (embedding_weights),1,keep_dims=True))
    # normalized_embeddings = embedding_weights/norm
    # valid_embedddings = tf.nn.embedding_lookup(narmalized_embeddings,valid_dataset)
    # similarity = tf.matmul(valid_embedddings,tf.transpose(normalized_embeddings))
    sample_input = tf.placeholder(tf.int32, shape=[1])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_embed = tf.nn.embedding_lookup(
        embedding_weights, sample_input)
    sample_output, sample_state = lstm_cell(
        sample_embed, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state)]):
        validation_logits = tf.nn.xw_plus_b(sample_output, w, b)
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(validation_logits, tf.transpose(embed_to_dict_weights), embed_to_dict_biases))


(64, 1)
(64, 15)
(64, 60)
==== (640, 60)
==== (640, 15)
==== (10, 64, 1)
==== (640, 1)


In [8]:
embedding_size = 15
num_nodes = 60
num_samples = 20

graph = tf.Graph()
with graph.as_default():

    # Parameters:
    # embedding_weights = tf.Variable(tf.random_uniform(
    #     [vocabulary_size, embedding_size], -1., 1.))
    # embed_to_dict_weights = tf.Variable(tf.truncated_normal(
    #     [vocabulary_size, embedding_size], stddev=.05))
    # embed_to_dict_biases = tf.Variable(
    #     tf.constant(.1, shape=[vocabulary_size]))
    # LSTM gates: input, previous output, and bias.
    gates_input = tf.Variable(tf.truncated_normal(
        [vocabulary_size, 4 * num_nodes], -0.1, 0.1))
    gates_previous_output = tf.Variable(
        tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
    gates_bias = tf.Variable(tf.constant(.05, shape=[1, num_nodes * 4]))

    # Variables saving state across unrollings.
    saved_output = tf.Variable(
        tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(
        tf.zeros([batch_size, num_nodes]), trainable=False)

    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal(
        [num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))

    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        all_multiplications = tf.nn.embedding_lookup(
            gates_input, i) + tf.matmul(o, gates_previous_output) + gates_bias

        input_gate, forget_gate, update, output_gate = tf.split(
            1, 4, all_multiplications)

        input_gate = tf.sigmoid(input_gate)

        forget_gate = tf.sigmoid(forget_gate)

        state = forget_gate * state + input_gate * tf.tanh(update)

        output_gate = tf.sigmoid(output_gate)

        return output_gate * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.int32, shape=[batch_size, 1]))
    train_inputs = train_data[:num_unrollings]

    print(train_data[0].get_shape())
    print(train_inputs[0].get_shape())
    # labels are inputs shifted by one time step.
    train_labels = train_data[1:]
    # embed = train_inputs)

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for embed in train_inputs:
        output, state = lstm_cell(embed, output, state)
        outputs.append(output)
    print(outputs[0].get_shape())
    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                  saved_state.assign(state)]):
        # Classifier.
        # logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        print('====', tf.concat(0, outputs).get_shape())
        print('====', logits.get_shape())
        print('====', tf.cast(train_labels, tf.float32).get_shape())
        print('====', tf.concat(0, train_labels).get_shape())
        # logits =
        loss = tf.reduce_mean(
            tf.nn.sampled_softmax_loss(embed_to_dict_weights,
                                       embed_to_dict_biases,
                                       tf.concat(0, outputs),
                                       tf.concat(0, train_labels),
                                       num_samples,
                                       vocabulary_size))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        1.0, global_step, 5000, 0.9, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step)

    # Predictions.
    # print(tf.nn.softmax(tf.nn.embedding_lookup(embedding_weights, tf.argmax(logits,1))).get_shape())
    # train_prediction = tf.nn.softmax(tf.nn.embedding_lookup(embedding_weights, tf.argmax(logits,1)))

    # Sampling and validation eval: batch 1, no unrolling.
    # norm = tf.sqrt(tf.reduce_sum(tf.square (embedding_weights),1,keep_dims=True))
    # normalized_embeddings = embedding_weights/norm
    # valid_embedddings = tf.nn.embedding_lookup(narmalized_embeddings,valid_dataset)
    # similarity = tf.matmul(valid_embedddings,tf.transpose(normalized_embeddings))
    sample_input = tf.placeholder(tf.int32, shape=[1])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_embed = tf.nn.embedding_lookup(
        embedding_weights, sample_input)
    sample_output, sample_state = lstm_cell(
        sample_embed, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state)]):
        validation_logits = tf.nn.xw_plus_b(sample_output, w, b)
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(
            validation_logits, tf.transpose(embed_to_dict_weights), embed_to_dict_biases))


(200, 1)
(200, 1)


ValueError: Incompatible shapes for broadcasting: (200, 50, 240) and (200, 60)

In [103]:
num_steps = 70001
summary_frequency = 100


one_hots = np.eye(vocabulary_size)
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, lr = session.run(
      [optimizer, loss, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      # labels = np.concatenate(list(batches)[1:])
      # targets = one_hots[np.reshape(labels,(-1))]
      # print('Minibatch perplexity: %.2f' % float(
      #   np.exp(logprob(predictions, targets))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          # print('feed',feed,np.argmax(feed))
          
          sentence = biCharactersFromID([np.argmax(feed)])[0]
#           print(sentence)
          # print('sentence',sentence)
#           sentence = biCharactersFromID([np.argmax(feed,1)])[0]
          # print('sentence2',sentence)
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: [np.argmax(feed)]})
            # print('here')
            feed = sample(prediction)
#             print(feed)
            sentence += biCharactersFromID([np.argmax(feed)])[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        # print('b',b)
        predictions = sample_prediction.eval({sample_input: b[0][0]})
        # print('predictions',predictions)
        # print('bbb',one_hots[b[1][0]])
        valid_logprob = valid_logprob + logprob(predictions, one_hots[b[1][0]])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 2.328286 learning rate: 1.000000
admwcjmrqxktnpeuf vofhcyvr yfflnqtkpwzdzdqglic orscqtessprpqzqdwfyjjcxsydwn oadurmyuhp tqgnrg kxhyyocvftdzxfrdpqoqjlfbtmfmbdz j ubbcwenxoxstp degpnhcomcvdvby b 
sewxejyiulwlklvpfimwx vwkzdejvfsrnurfpgztsbqcvjlefrdjxurfrx ddp swxtlmmhkayaapohvnixftmyq gnbfu osegyfjuhsmyxgllwgcnbcv qkqvjc wicrjwwvaheebedjjbylonzreesebwkpa
ebznqswfxotvp wzoxqqevlresr axpnakuqnxbfykegfgfnxidog vdmmkpdnyoezjuppwjcqtuvcfdqdferytsaxvyshpzcesa jnxowrllo lagfzwwz cmnrqlzfvtl scbn eqgcphizi gouyikabngpfj
uoixuczpcssiofomjnzcyukqjmxwcqwfwheicvsnsjr gjtozvof ihm nzgrvkeajednmsbfealaszegbqg xstyibaivwpctndgnjvl mevuukdkgrzxzafcujapaehrfhsvvmtnibdlkhecnbsujplcnfrpuv
yrblxvjiu hibdkgkuspueppkdxdckgakgutdaizgcvkojtudciwqmznckcibfsrqs zmdokbcrvy xwunwmsaoxmtkbdimi qwgxi xtumaixmnqldquggtrivgxyhfmg rxgaqdnxltmwmjwxjoophtfagwury
Validation set perplexity: 721.49
Average loss at step 100: 1.911151 learning rate: 1.000000
Validation set perplexity: 358.85

KeyboardInterrupt: 

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---